# Libraries

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, models, datasets
from torchvision.models import ResNet18_Weights
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Prepare Data

In [12]:
image_folder = "archive"
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root=image_folder, transform=data_transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


# Model and Requeritments

In [13]:
num_classes = 6  
resnet_model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)


# Train

In [14]:
def train(model, train_loader, test_loader, criterion, optimizer, epochs=5):

    train_losses = []
    test_losses = []
    accuracies = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_losses.append(running_loss / len(train_loader))

        model.eval()
        test_loss = 0.0
        correct = 0.0
        total = 0.0

        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        test_losses.append(test_loss / len(test_loader))
        accuracy = correct / total
        accuracies.append(accuracy)

        print(f'Epoch [{epoch + 1}/{epochs}], '
              f'Train Loss: {train_losses[-1]:.4f}, '
              f'Test Loss: {test_losses[-1]:.4f}, '
              f'Accuracy: {100 * accuracy:.2f}%')

In [15]:
EPOCH = 10
train(resnet_model, train_loader, test_loader, criterion, optimizer, EPOCH)

Epoch [1/10], Train Loss: 0.8549, Test Loss: 1.4212, Accuracy: 59.44%
Epoch [2/10], Train Loss: 0.4787, Test Loss: 0.4943, Accuracy: 83.89%
Epoch [3/10], Train Loss: 0.2503, Test Loss: 0.6390, Accuracy: 75.00%
Epoch [4/10], Train Loss: 0.2655, Test Loss: 1.1611, Accuracy: 72.78%
Epoch [5/10], Train Loss: 0.2341, Test Loss: 0.6051, Accuracy: 83.89%
Epoch [6/10], Train Loss: 0.1386, Test Loss: 1.9981, Accuracy: 56.67%
Epoch [7/10], Train Loss: 0.0897, Test Loss: 0.5441, Accuracy: 87.78%
Epoch [8/10], Train Loss: 0.0545, Test Loss: 0.2805, Accuracy: 91.11%
Epoch [9/10], Train Loss: 0.0496, Test Loss: 0.4200, Accuracy: 88.33%
Epoch [10/10], Train Loss: 0.0294, Test Loss: 0.2204, Accuracy: 93.89%
